In [3]:
import sys
sys.path.append(r"D:\BEHAVIOR-1K\asset_pipeline")
import b1k_pipeline.utils

In [7]:
import glob, json, os
from tqdm.notebook import tqdm

files = glob.glob(r"D:\BEHAVIOR-1K\asset_pipeline\cad\*\*\artifacts\collision_selection.json")
for fn in tqdm(files):
    # Read selection
    with open(fn, "r") as f:
        selection = json.load(f)
        
    # Remove None
    new_selection = {k: v for k,v in selection.items() if v is not None}
    removed = set(selection.keys()) - set(new_selection.keys())
    
    # Rewrite selection
    with open(fn, "w") as f:
        json.dump(new_selection, f, indent=4, sort_keys=True)
        
    # Add complaints
    if removed:
        target_dir = os.path.dirname(os.path.dirname(fn))
        complaints_file = os.path.join(target_dir, "complaints.json")
        complaints = []
        if os.path.exists(complaints_file):
            with open(complaints_file, "r") as f:
                complaints = json.load(f)
        for r in sorted(removed):
            parsed_name = b1k_pipeline.utils.parse_name(r)
            object_key = parsed_name.group("category") + "-" + parsed_name.group("model_id")
            complaints.append({
                "object": object_key,
                "message": "Was at least one of the collision mesh candidates acceptable?",
                "complaint": "Automated complaint: None was marked in original pass - please review.",
                "processed": False
            })

        with open(complaints_file, "w") as f:
            json.dump(complaints, f, indent=4)

  0%|          | 0/1620 [00:00<?, ?it/s]